In [26]:
import re
from konlpy.tag import Okt
from collections import Counter
print("임포트 완료")

임포트 완료


In [27]:
text = '임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어.'
text

'임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어.'

In [28]:
# 전처리 - 특수문자 제거
reg = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')
text = reg.sub('', text)
print(text)

임금님 귀는 당나귀 귀 임금님 귀는 당나귀 귀 실컷 소리치고 나니 속이 확 뚫려 살 것 같았어


In [29]:
# 형태소 분석기를 통한 토큰화
okt=Okt()
tokens = okt.morphs(text)
print(tokens)

['임금님', '귀', '는', '당나귀', '귀', '임금님', '귀', '는', '당나귀', '귀', '실컷', '소리', '치고', '나니', '속이', '확', '뚫려', '살', '것', '같았어']


In [30]:
# python Counter() 섭클레스 사용, 빈도를 세고 빈도가 높은 순서로 정렬
vocab = Counter(tokens)
print(vocab)

Counter({'귀': 4, '임금님': 2, '는': 2, '당나귀': 2, '실컷': 1, '소리': 1, '치고': 1, '나니': 1, '속이': 1, '확': 1, '뚫려': 1, '살': 1, '것': 1, '같았어': 1})


In [31]:
# 빈도수 리턴
vocab['임금님']

2

In [32]:
# 빈도수 상위 5개의 단어만 저장
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

[('귀', 4), ('임금님', 2), ('는', 2), ('당나귀', 2), ('실컷', 1)]


In [33]:
# 상위 5개의 단어에 인덱스 부여
word2idx={word[0] : index+1 for index, word in enumerate(vocab)}
print(word2idx)

{'귀': 1, '임금님': 2, '는': 3, '당나귀': 4, '실컷': 5}


In [34]:
def one_hot_encoding(word, word2index):
       one_hot_vector = [0]*(len(word2index))
       index = word2index[word]
       one_hot_vector[index-1] = 1
       return one_hot_vector

In [36]:
one_hot_encoding('임금님', word2idx)

[0, 1, 0, 0, 0]

- keras를 통한 원-핫 인코딩

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
print("임포트 완료")

임포트 완료


In [38]:
text = [['강아지', '고양이', '강아지'],['애교', '고양이'], ['컴퓨터', '노트북']]
text

[['강아지', '고양이', '강아지'], ['애교', '고양이'], ['컴퓨터', '노트북']]

In [40]:
t = Tokenizer()
t.fit_on_texts(text)
print(t.word_index) # 각 단어에 대한 인코딩 결과 출력.

{'강아지': 1, '고양이': 2, '애교': 3, '컴퓨터': 4, '노트북': 5}


In [41]:
# 단어장 크기 저장
vocab_size = len(t.word_index) + 1

- 0번 단어가 특별 토큰으로 단어장에 추가되는 경우가 많음
- 0번은 패딩 토큰
- 그래서 vocab_size에 +1 해줌 (0번 토큰을 만들고 비워둠)

In [43]:
# 텍스트를 정수 시퀀스로 변환 (단어장에 속한 단어의 경우 정수로 변환)
sub_text = ['강아지', '고양이', '강아지', '컴퓨터']
encoded = t.texts_to_sequences([sub_text])
print(encoded)

[[1, 2, 1, 4]]


In [45]:
one_hot = to_categorical(encoded, num_classes = vocab_size)
print(one_hot)
# 0 1 2 3 4 5 클래스

[[[0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0.]]]


# Word Embedding

## 1. Word2Vec
- 분포가설
    - 곁에 오는 단어들을 보면 그 단어를 알 수 있다.
    - 비슷한 문맥에서 같이 등장하는 경향이 있는 단어들은 비슷한 의미를 가진다.
- Continuous Bag of Words (CBoW)
    - 중심 단어를 예측하기 위해 주변 단어를 입력으로 사용
    - 예문: "I like natural language processing"
        - {"i", "like", "language", "processing"}으로부터 "natural"을 예측
        - 중심단어(center word): "natural"
        - 주변단어(context word): 그외 예측에 사용되는 단어들
        - 윈도우(window): 중심단어를 예측하기 위해 볼 주변 단위의 범위
        - 데이터셋 구성 형태: **((주변1, 주변2), 중심)**
            - ((like), I), ((I, natural), like), ((like, language), natural), ((natural, processing), language), ((language), processing)
    - Out-of-vocabulary(단어 집합에 없는 단어), Polysemy(다의어), 문맥파악의 한계점 존재
- Skip-gram
    - 중심 단어를 예측하기 위해 주변 단어를 입력으로 사용(CBoW와 메커니즘 자체는 동일)
    - 데이터셋 구성 형태: **(중심, 주변)**
        - (i, like) (like, I), (like, natural), (natural, like), (natural, language), (language, natural), (language, processing), (processing, language)
- Negative sampling
    - 대체적으로 SGNS(Skip-Gram with Negative Sampling)을 사용.
    - 중심단어와 주변단어를 동시에 입력받아 두 단어가 정말 이웃 관계면 1 또는 0을 출력
    - 예문: "Thou shalt not make a machine in the likeness of a human mind"
| Input   | Output  | Target  |
| -------- | -------- | -------- |
| not      | Thou     | 1     |
| not      | shalt    | 1     |
| not      | make     | 1     |
| not      | a        | 1     |
| make     | shalt    | 1     |
| make     | not      | 1     |
| make     | a        | 1     |
| make     | machine  | 1     |
| .......  |  ....... | .......  |

    - 랜덤으로 단어장에 있는 아무 단어나 가져와 target word로 하는 거짓 데이터셋을 만들어 0으로 레이블
    - 이진 문제로 간주 - 출력층의 시그모이드 함수 - 레이블로부터 오차 역전파

### 1-1. Word2Vec Practice

In [46]:
import nltk
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Downloading package abc to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package abc is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
from nltk.corpus import abc
corpus = abc.sents()

In [49]:
print(corpus[:3])

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ['In', 'one', 'of', 'the', 'letters', 'Mr', 'Howard', 'asks', 'AWB', 'managing', 'director', 'Andrew', 'Lindberg', 'to', 'remain', 'in', 'close', 'contact', 'with', 'the', 'Government', 'on', 'Iraq', 'wheat', 'sales', '.']]


In [50]:
print('코퍼스의 크기 :',len(corpus))

코퍼스의 크기 : 29059


In [52]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
print('모델 학습 완료!')

모델 학습 완료!


- vector_size: 학습 후 임베딩 벡터 차원
- window: 컨텍스트 윈도우 크기
- min_count: 단어 최소 빈도수 제한 (빈도 적은 단어들은 학습에서 제외)
- workers: 학습을 위한 프로세스 수
- sg: 0(CBow), 1(Skip-gram)

In [53]:
model_result = model.wv.most_similar('man')
print(model_result)

[('woman', 0.9233394265174866), ('skull', 0.9110252261161804), ('Bang', 0.9056580066680908), ('asteroid', 0.9052032232284546), ('third', 0.9020134806632996), ('baby', 0.8994045853614807), ('dog', 0.898598849773407), ('bought', 0.8975046277046204), ('rally', 0.8912537097930908), ('disc', 0.8889098167419434)]


In [54]:
import os
from gensim.models import KeyedVectors
# 모델 저장
os.makedirs('./word_embedding/', exist_ok=True)
model.wv.save_word2vec_format('./word_embedding/w2v')

In [55]:
# 모델 불러오기
loaded_model = KeyedVectors.load_word2vec_format('./word_embedding/w2v')
print('모델 load 완료!')

모델 load 완료!


In [56]:
model_result = loaded_model.most_similar('man')
print(model_result)

[('woman', 0.9233394265174866), ('skull', 0.9110252261161804), ('Bang', 0.9056580066680908), ('asteroid', 0.9052032232284546), ('third', 0.9020134806632996), ('baby', 0.8994045853614807), ('dog', 0.898598849773407), ('bought', 0.8975046277046204), ('rally', 0.8912537097930908), ('disc', 0.8889098167419434)]


#### Out of Vocabulary(OOV) 문제

In [57]:
# 단어장에 없는 단어 입력시 에러
loaded_model.most_similar('overacting')

KeyError: "Key 'overacting' not present in vocabulary"

In [58]:
# memory의 오타 가정
loaded_model.most_similar('memorry')

KeyError: "Key 'memorry' not present in vocabulary"

## 2. FastText
- 단어 내부의 subwords들을 학습
- 예문: "partial"
    - n = 3, <pa, par, art, rti, tia, ial, al>, \<partial>
    - n = 3 ~ 6, <pa, par, art, rti, ita, ial, al>, <par, arti, rtia, tial, ial>, <part, ...중략... , \<partial>

In [59]:
from gensim.models import FastText
fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)
print('FastText 학습 완료!')

FastText 학습 완료!


In [61]:
# Word2Vec에서 에러 발생했던 단어1
fasttext_model.wv.most_similar('overacting')

[('extracting', 0.9394649267196655),
 ('resolving', 0.9356498718261719),
 ('fluctuating', 0.932230532169342),
 ('attracting', 0.9315232634544373),
 ('declining', 0.92936772108078),
 ('lifting', 0.9286537766456604),
 ('reflecting', 0.9280219674110413),
 ('boosting', 0.9263381361961365),
 ('mounting', 0.9263135194778442),
 ('contracting', 0.9252020716667175)]

In [62]:
# Word2Vec에서 에러 발생했던 단어2
fasttext_model.wv.most_similar('memoryy')

[('memory', 0.9533040523529053),
 ('mechanisms', 0.8768883347511292),
 ('mechanism', 0.8692528009414673),
 ('musical', 0.8585295081138611),
 ('music', 0.8582504391670227),
 ('intelligence', 0.8534806966781616),
 ('basic', 0.8524240255355835),
 ('intermediate', 0.8434807658195496),
 ('mechanical', 0.8363848328590393),
 ('intercourse', 0.8351597785949707)]

## 3. Glove
- 중심 단어 벡터와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 빈도의 로그값이 되도록 만드는 것
- 전체 코퍼스에서의 동시 등장 빈도의 로그값과 중심 단어 벡터와 주변 단어 벡터의 내적값의 차이가 최소화되도록 두 벡터의 값을 학습하는 것

In [63]:
import gensim.downloader as api
glove_model = api.load("glove-wiki-gigaword-50")  # glove vectors 다운로드
glove_model.most_similar("dog")  # 'dog'과 비슷한 단어 찾기

[=================---------------------------------] 35.5% 23.4/66.0MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 66.0/66.0MB downloaded


[('cat', 0.9218004941940308),
 ('dogs', 0.8513158559799194),
 ('horse', 0.7907583117485046),
 ('puppy', 0.7754921317100525),
 ('pet', 0.7724708318710327),
 ('rabbit', 0.7720813751220703),
 ('pig', 0.7490060925483704),
 ('snake', 0.7399188280105591),
 ('baby', 0.7395570278167725),
 ('bite', 0.7387937307357788)]

In [64]:
glove_model.most_similar('overacting')

[('impudence', 0.7842012643814087),
 ('puerile', 0.7816032767295837),
 ('winningly', 0.7644237875938416),
 ('grossness', 0.7576098442077637),
 ('deconstructions', 0.748936653137207),
 ('over-the-top', 0.7460805177688599),
 ('buffoonery', 0.7460456490516663),
 ('impetuosity', 0.7415392398834229),
 ('sophomoric', 0.7369617819786072),
 ('zaniness', 0.7353196740150452)]

In [65]:
glove_model.most_similar('memoryy')

KeyError: "Key 'memoryy' not present in vocabulary"